In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# change path if necessary
import sys
my_path = r'D:\Documents\etudes\epfl\MA1\cours\MachineLearning\Project1' # to be adapated
sys.path.insert(0,my_path + r'\code\COMMON')

# imports
import numpy as np 
import matplotlib.pyplot as plt

from implementations import *
from labels import idx_2labels
from costs import *
from optimize_hyperparams import *
from cross_validation import *
from step_wise import *

# Build dataset

In [2]:
from proj1_helpers import load_csv_data 

# load raw data
y_raw, input_data_raw, ids = load_csv_data(my_path + r'\data\train.csv', sub_sample=False)

In [3]:
from outliers import handle_outliers

# handle outliers
X_raw, y = handle_outliers(input_data_raw, y_raw, -999, 'median')

# set y in {0,1} instead of {-1,1}
y[np.where(y==-1)]=0

-999 are replaced by the median value of the feature


In [4]:
# get feature names 
all_features_raw = list(np.genfromtxt(my_path + r'/data/train.csv', delimiter=",", dtype=str, max_rows = 1)[2:])

In [6]:
from extend_features import extend_features

# feature degree
degree = 1

# extend feature set
all_candidates, features = extend_features(X_raw, all_features_raw, degree, is_add_log = True)
print(all_candidates.shape)

---------------------------
Features have been set to the power(s): [1]
4 logarithmic features have been added.
Data have been standardized.
---------------------------
(250000, 34)


# Feature Selection through step-wise

In [7]:
# model parameters
model = dict()
model['method'] = 'lr' # 'lrr' if regularized with a penalization 'lambda_' otherwise 'lr' 
model['loss'] = 'loglikelihood'

# model hyperparameter
model['gamma'] = 1e-5

# other
model['method_minimization'] = 'gd' # 'gd' (gradient_descent) or 'newton' (newton descent leads to non invertible matrice "S" which is too big)
model['threshold'] = 1e-2
model['max_iters'] = 10000
model['debug_mode'] = 0

# R2 type
R2_method = 'McFadden' 

# estimate R2 error through cross validation (1 or 0)
cv = 1
model['k_fold'] = 10

In [8]:
# step-wise
best_R2adj, idx_features = stepwise(model, R2_method, all_candidates, features, y, cv)

Number of input features (all the candidates): 34
Number samples: 250000
-------------------------------------------------
Feature chosen:  DER_mass_transverse_met_lep_power_1 (index : 1 )
-------------------------------------------------
Feature chosen:  PRI_tau_pt_power_1 (index : 13 )
-------------------------------------------------
Feature chosen:  DER_deltaeta_jet_jet_power_1 (index : 4 )


KeyboardInterrupt: 

In [ ]:
# display selected features
results_r2_stepwise(best_R2adj[:len(best_R2adj)-1], idx_features)

In [ ]:
# save the selected features
Xsw = all_candidates(:, idx_features)

# Optimization of lambda through simple cross validation

In [ ]:
# optimization parameters
lambda_min = -10 
lambda_max = 1
lambda_steps = 10
model['k_fold'] = 10

# optimize lambda_
w_opt, loss_tr, loss_te, lambda_opt, success_rate = optimize_lambda(y, Xsw, lambda_min, lambda_max, lambda_steps, arg_rr, debug_mode=1)